In [1]:
import sympy
from sympy.abc import x,y,z
from cmath import pi

# Rotations tools

In [2]:
class Rotations:
    def __init__(self):
        self.alpha  = sympy.Symbol('alpha')
        self.beta  = sympy.Symbol('beta')
        self.gamma = sympy.Symbol('gamma')
        self.theta = sympy.Symbol('theta')
        self.zRotM = sympy.Matrix([
            [sympy.cos(self.theta),-sympy.sin(self.theta), 0],
            [sympy.sin(self.theta), sympy.cos(self.theta), 0],
            [0                    , 0                    , 1]
        ])
        self.yRotM = sympy.Matrix([
            [sympy.cos(self.theta), 0,-sympy.sin(self.theta)],
            [0                    , 1, 0                    ],
            [sympy.sin(self.theta), 0, sympy.cos(self.theta)]
        ])
        self.xRotM = sympy.Matrix([
            [1, 0                    , 0                    ],
            [0, sympy.cos(self.theta),-sympy.sin(self.theta)],
            [0, sympy.sin(self.theta), sympy.cos(self.theta)]
        ])
    
    def eulerToRotMSequence(self,sequence:str, a,b,c):
        if len(sequence) != 3: return None
        rotSymbols = [a,b,c]
        rotM = sympy.eye(3)
        for axis,symbol in zip(list(sequence),rotSymbols):
            if axis is "x":
                rotM = rotM*self.xRotM.subs(self.theta,symbol)
            elif axis is "y":
                rotM = rotM*self.yRotM.subs(self.theta,symbol)
            elif axis is "z":
                rotM = rotM*self.zRotM.subs(self.theta,symbol)
        return rotM
    
    def zyxToRotM(self,z,y,x):
        return self.eulerToRotMSequence("zyx",z,y,x)

# Two-Link Planar 

In [3]:
class TwoLinkPlanar:

    def __init__(self, a1=None, a2=None) -> None:
        self.a1 = sympy.Symbol('a_1') if a1 is None else a1
        self.a2 = sympy.Symbol('a_2') if a2 is None else a2
        self.q1 = sympy.Symbol('q_1')
        self.q2 = sympy.Symbol('q_2')
        self.x  = x
        self.y  = y
        self.theta = sympy.Symbol('theta')
        self.directLambda  = lambda q1,q2: None
        self.inverseLambda = lambda x,y: None
        self._resetSymbolsAndLambdas()

    def directKinSymbols(self):
        x_dk     = self.a1*sympy.cos(self.q1) + self.a2*sympy.cos(self.q1+self.q2)
        y_dk     = self.a1*sympy.sin(self.q1) + self.a2*sympy.sin(self.q1+self.q2)
        theta_dk = self.q1 + self.q2
        return sympy.Matrix((x_dk,y_dk,theta_dk))

    def inverseKinSymbols(self):
        q2_ik = sympy.acos((self.x**2 + self.y**2 - self.a1**2 - self.a2**2)/(2*self.a1*self.a2))
        q1_ik = sympy.atan2(self.y,self.x) - sympy.atan2(self.a2*sympy.sin(q2_ik), self.a1 + self.a2*sympy.cos(q2_ik))
        return sympy.Matrix((q1_ik,q2_ik))

    def setLengths(self,a1,a2):
        self.a1 = a1
        self.a2 = a2
        self._resetSymbolsAndLambdas()
    
    def setJointCoordinates(self,joint1,joint2):
        self.q1 = joint1
        self.q2 = joint2
        self._resetSymbolsAndLambdas()

    def setCartesianCoordinates(self,coord1,coord2):
        self.x = coord1
        self.y = coord2
        self._resetSymbolsAndLambdas()
    
    def _resetSymbolsAndLambdas(self):
        # Compute simbolic kinematics
        self.directSym  = self.directKinSymbols()
        self.inverseSym = self.inverseKinSymbols()
        # Compute lambda kinematic functions 
        try:
            self.directLambda  = sympy.lambdify((self.q1,self.q2),self.directSym)
            self.inverseLambda = sympy.lambdify((self.x,self.y),self.inverseSym)
        except SyntaxError:
            self.directLambda  = lambda q1,q2: None
            self.inverseLambda = lambda x,y: None

## Examples

In [4]:
twp = TwoLinkPlanar(2,3)
twp.directKinSymbols()

Matrix([
[2*cos(q_1) + 3*cos(q_1 + q_2)],
[2*sin(q_1) + 3*sin(q_1 + q_2)],
[                    q_1 + q_2]])

In [5]:
twp.directLambda(pi/2,pi/2)

array([[-3.        ],
       [ 2.        ],
       [ 3.14159265]])

In [6]:
twp.inverseLambda(-1,2)

array([[0.46364761],
       [2.30052398]])

# Two-Link Planar on rotatory base

In [7]:
class TwoLinkAndBase():

    def __init__(self, a1=None, a2=None) -> None:
        self.a1 = sympy.Symbol('a_1') if a1 is None else a1
        self.a2 = sympy.Symbol('a_2') if a2 is None else a2
        self.q1 = sympy.Symbol('q_1')
        self.q2 = sympy.Symbol('q_2')
        self.q3 = sympy.Symbol('q_3')
        self.x = x
        self.y = y
        self.z = z
        self._r = sympy.sqrt(self.x**2 + self.y**2)
        self.twoLink = TwoLinkPlanar(a1,a2)
        self.twoLink.setCartesianCoordinates(self._r, self.z)
        self.twoLink.setJointCoordinates(self.q2,self.q3)
        # Initialize symbolic kinematics
        self.directSym  = self.directKinSymbols()
        self.inverseSym = self.inverseKinSymbols()   
        # Initialize lambdas
        self.directLambda  = lambda q1,q2,q3: None
        self.inverseLambda = lambda x, y, z : None
        # Compute symbolic kinematics and lambda functions
        self._resetSymbolsAndLambdas()

    def setLengths(self,a1,a2):
        self.a1 = a1
        self.a2 = a2
        self._resetSymbolsAndLambdas()
    
    def setJointCoordinates(self,joint1,joint2,joint3):
        self.q1 = joint1
        self.q2 = joint2
        self.q3 = joint3
        self._resetSymbolsAndLambdas()

    def setCartesianCoordinates(self,coord1,coord2,coord3):
        self.x = coord1
        self.y = coord2
        self.z = coord3
        self._resetSymbolsAndLambdas()
        
    def directKinSymbols(self):
        r_dk,z_dk,pitch_dk = self.twoLink.directKinSymbols()
        x_dk = r_dk*sympy.cos(self.q1)
        y_dk = r_dk*sympy.sin(self.q1)
        yaw_dk = self.q1
        roll_dk = 0
        return sympy.Matrix((x_dk,y_dk,z_dk,yaw_dk,pitch_dk,roll_dk))

    def directKinHomoTSymbols(self):
        x_dk,y_dk,z_dk,yaw_dk,pitch_dk,roll_dk = self.directKinSymbols()
        rotM = Rotations().zyxToRotM(yaw_dk,pitch_dk,roll_dk)
        translation = sympy.Matrix((x_dk,y_dk,z_dk))
        return sympy.Matrix((
            (rotM            , translation    ),
            (sympy.zeros(1,3), sympy.ones(1,1))
        ))

    def inverseKinSymbols(self):
        q1_ik = sympy.atan2(self.y,self.x)
        q2_ik,q3_ik = self.twoLink.inverseKinSymbols()
        return sympy.Matrix((q1_ik,q2_ik,q3_ik))
        
    def _resetSymbolsAndLambdas(self):
        # Compute simbolic kinematics
        self.directSym  = self.directKinSymbols()
        self.inverseSym = self.inverseKinSymbols()
        # Compute lambda kinematic functions 
        try:
            self.directLambda  = sympy.lambdify((self.q1,self.q2,self.q3),self.directSym)
            self.inverseLambda = sympy.lambdify((self.x, self.y, self.z), self.inverseSym)
        except SyntaxError:
            self.directLambda  = lambda q1,q2,q3: None
            self.inverseLambda = lambda x, y, z : None

## Examples

In [8]:
tlb = TwoLinkAndBase(4,3)
tlb.directKinHomoTSymbols()

Matrix([
[cos(q_1)*cos(q_2 + q_3), -sin(q_1), -sin(q_2 + q_3)*cos(q_1), (4*cos(q_2) + 3*cos(q_2 + q_3))*cos(q_1)],
[sin(q_1)*cos(q_2 + q_3),  cos(q_1), -sin(q_1)*sin(q_2 + q_3), (4*cos(q_2) + 3*cos(q_2 + q_3))*sin(q_1)],
[         sin(q_2 + q_3),         0,           cos(q_2 + q_3),            4*sin(q_2) + 3*sin(q_2 + q_3)],
[                      0,         0,                        0,                                        1]])

In [9]:
tlb.inverseKinSymbols()

Matrix([
[                                                                                                              atan2(y, x)],
[atan2(z, sqrt(x**2 + y**2)) - atan2(3*sqrt(1 - (x**2/24 + y**2/24 + z**2/24 - 25/24)**2), x**2/8 + y**2/8 + z**2/8 + 7/8)],
[                                                                                acos(x**2/24 + y**2/24 + z**2/24 - 25/24)]])

In [10]:
tlb.directLambda(0,pi/4,pi/4)

array([[2.82842712],
       [0.        ],
       [5.82842712],
       [0.        ],
       [1.57079633],
       [0.        ]])

In [11]:
tlb.inverseLambda(4,0,4)

array([[0.        ],
       [0.25338261],
       [1.27482752]])

# 3DOF Rotational wrist 

In [30]:
import symbol


class RotationalWrist:
    def __init__(self, sequence:str) -> None:
        self.q1 = sympy.Symbol('q_1')
        self.q2 = sympy.Symbol('q_2')
        self.q3 = sympy.Symbol('q_3')
        self.yaw   = sympy.Symbol('gamma')
        self.pitch = sympy.Symbol('beta')
        self.roll  = sympy.Symbol('alpha')
        self.sequence = sequence
        self.singularities = {}
        # Initialize symbolic kinematics
        self.directSym  = self.directKinHomoTSymbols()
        self.inverseSym = self.inverseKinSymbols()   
        # Initialize lambdas
        self.directLambda  = lambda q1,q2,q3: None
        self.inverseLambda = lambda x, y, z : None
        # Cmpute symbolic kinematics and lambda functions
        self._resetSymbolsAndLambdas()

    def setJointCoordinates(self,joint1,joint2,joint3):
        self.q1 = joint1
        self.q2 = joint2
        self.q3 = joint3
        self._resetSymbolsAndLambdas()

    def setCartesianCoordinates(self,coord1,coord2,coord3):
        self.yaw   = coord1
        self.pitch = coord2
        self.roll  = coord3
        self._resetSymbolsAndLambdas()

    def directKinHomoTSymbols(self):
        self.rotM = Rotations().eulerToRotMSequence(self.sequence,self.q1,self.q2,self.q3)
        return sympy.Matrix((
            (self.rotM       , sympy.zeros(3,1) ),
            (sympy.zeros(1,3), sympy.ones(1,1)  )
        ))

    def inverseKinSymbols(self):
        rotM = Rotations().eulerToRotMSequence(self.sequence,self.yaw,self.pitch,self.roll)
        if self.sequence is "xyx":
            q1_dk = sympy.atan2(-rotM[1,0], rotM[2,0])
            q2_dk = sympy.acos(  rotM[0,0])
            q3_dk = sympy.atan2( rotM[0,1], rotM[0,2])
            self.singularities.update({self.pitch:0})
        else:
            return None

        return sympy.Matrix((q1_dk,q2_dk,q3_dk))

    def direct(self,q1,q2,q3):
        return self.directLambda(q1,q2,q3)
    
    def inverse(self,y,p,r):
        return self.inverseLambda(y,p,r)

    def _resetSymbolsAndLambdas(self):
        # Compute simbolic kinematics
        self.directSym  = self.directKinHomoTSymbols()
        self.inverseSym = self.inverseKinSymbols()
        print(f"Singularities in {self.singularities}")
        # Compute direct kinematic symbols
        dkSymbols = self._orderedSymbolsFromExpr(self.directSym)
        # Compute lambda kinematic functions 
        try:
            self.directLambda  = sympy.lambdify(dkSymbols,self.directSym)
            self.inverseLambda = sympy.lambdify((self.yaw, self.pitch, self.roll), self.inverseSym)
        except SyntaxError:
            self.directLambda  = lambda q1,q2,q3: None
            self.inverseLambda = lambda x, y, z : None
    
    def _orderedSymbolsFromExpr(self,expression):
        names = [sym.name for sym in expression.free_symbols]
        names.sort()
        return [sympy.Symbol(n) for n in names]

## Examples

In [31]:
rw = RotationalWrist("xyx")

Singularities in {beta: 0}


In [32]:
rw.directKinHomoTSymbols()

Matrix([
[          cos(q_2),                              -sin(q_2)*sin(q_3),                              -sin(q_2)*cos(q_3), 0],
[-sin(q_1)*sin(q_2), -sin(q_1)*sin(q_3)*cos(q_2) + cos(q_1)*cos(q_3), -sin(q_1)*cos(q_2)*cos(q_3) - sin(q_3)*cos(q_1), 0],
[ sin(q_2)*cos(q_1),  sin(q_1)*cos(q_3) + sin(q_3)*cos(q_1)*cos(q_2), -sin(q_1)*sin(q_3) + cos(q_1)*cos(q_2)*cos(q_3), 0],
[                 0,                                               0,                                               0, 1]])

In [33]:
rw.inverseKinSymbols()

Matrix([
[  atan2(sin(beta)*sin(gamma), sin(beta)*cos(gamma))],
[                                    acos(cos(beta))],
[atan2(-sin(alpha)*sin(beta), -sin(beta)*cos(alpha))]])

In [34]:
rw.inverse(0,0,0)

array([[ 0.        ],
       [ 0.        ],
       [-3.14159265]])

# Decoupled kinematics 6DOF

In [35]:
class Decoupled6DOF:
    def __init__(self,a1=None,a2=None) -> None:
        self.a1 = sympy.Symbol('a_1') if a1 is None else a1
        self.a2 = sympy.Symbol('a_2') if a2 is None else a2
        self.q1 = sympy.Symbol('q_1')
        self.q2 = sympy.Symbol('q_2')
        self.q3 = sympy.Symbol('q_3')
        self.q4 = sympy.Symbol('q_4')
        self.q5 = sympy.Symbol('q_5')
        self.q6 = sympy.Symbol('q_6')
        self.x = x
        self.y = y
        self.z = z
        self.yaw   = sympy.Symbol('gamma')
        self.pitch = sympy.Symbol('beta')
        self.roll  = sympy.Symbol('alpha')
        self.positionSide    = TwoLinkAndBase(a1,a2)
        self.positionSide.setCartesianCoordinates(self.x,self.y,self.z)
        self.positionSide.setJointCoordinates(self.q1,self.q2,self.q3)
        self.orientationSide = RotationalWrist("xyx")
        self.orientationSide.setCartesianCoordinates(self.yaw,self.pitch,self.roll)
        self.orientationSide.setJointCoordinates(self.q4,self.q5,self.q6)
        # Initialize symbolic kinematics
        self.directSym  = self.directKinHomoTSymbols()
        self.inverseSym = self.inverseKinSymbols()   
        # Initialize lambdas
        self.directLambda  = lambda q1,q2,q3: None
        self.inverseLambda = lambda x, y, z : None
        # Compute symbolic kinematics and lambda functions
        self._resetSymbolsAndLambdas()

    def directKinHomoTSymbols(self):
        return self.orientationSide.directKinHomoTSymbols() * self.positionSide.directKinHomoTSymbols()

    def inverseKinSymbols(self):
        positionKin    = self.positionSide.inverseKinSymbols()
        orientationFromPositon = sympy.Matrix(self.positionSide.directKinSymbols()[3:])
        orientationKin = self.orientationSide.inverseKinSymbols()
        orientationFromPositon = orientationFromPositon.subs(self.q1,-positionKin[0])
        orientationFromPositon = orientationFromPositon.subs(self.q2,-positionKin[1])
        orientationFromPositon = orientationFromPositon.subs(self.q3,-positionKin[2])
        
        return sympy.Matrix((positionKin,orientationFromPositon+orientationKin))
        
    def direct(self,q1,q2,q3,q4,q5,q6):
        return self.directLambda(q1,q2,q3,q4,q5,q6)

    def inverse(self,x,y,z,yaw,pitch,roll):
        return self.inverseLambda(x,y,z,yaw,pitch,roll)

    def _resetSymbolsAndLambdas(self):
        # Compute simbolic kinematics
        self.directSym  = self.directKinHomoTSymbols()
        # self.inverseSym = self.inverseKinSymbols()
        # Compute direct kinematic symbols
        dkSymbols = self._orderedSymbolsFromExpr(self.directSym)
        # Compute lambda kinematic functions 
        try:
            self.directLambda  = sympy.lambdify(dkSymbols,self.directSym)
            self.inverseLambda = sympy.lambdify((self.x, self.y, self.z, self.yaw, self.pitch, self.roll), self.inverseSym)
        except SyntaxError:
            self.directLambda  = lambda q1,q2,q3,q4,q5,q6: None
            self.inverseLambda = lambda x, y, z, yaw,pch,rol: None

    def _orderedSymbolsFromExpr(self,expression):
        names = [sym.name for sym in expression.free_symbols]
        names.sort()
        return [sympy.Symbol(n) for n in names]

## Examples

In [36]:
d6d = Decoupled6DOF(2,3)
d6d.directKinHomoTSymbols()

Singularities in {beta: 0}
Singularities in {beta: 0}
Singularities in {beta: 0}


Matrix([
[                                                                        -sin(q_1)*sin(q_5)*sin(q_6)*cos(q_2 + q_3) - sin(q_5)*sin(q_2 + q_3)*cos(q_6) + cos(q_1)*cos(q_5)*cos(q_2 + q_3),                                         -sin(q_1)*cos(q_5) - sin(q_5)*sin(q_6)*cos(q_1),                                                                           sin(q_1)*sin(q_5)*sin(q_6)*sin(q_2 + q_3) - sin(q_5)*cos(q_6)*cos(q_2 + q_3) - sin(q_2 + q_3)*cos(q_1)*cos(q_5),                                                                         -(2*sin(q_2) + 3*sin(q_2 + q_3))*sin(q_5)*cos(q_6) - (2*cos(q_2) + 3*cos(q_2 + q_3))*sin(q_1)*sin(q_5)*sin(q_6) + (2*cos(q_2) + 3*cos(q_2 + q_3))*cos(q_1)*cos(q_5)],
[(-sin(q_4)*sin(q_6)*cos(q_5) + cos(q_4)*cos(q_6))*sin(q_1)*cos(q_2 + q_3) + (-sin(q_4)*cos(q_5)*cos(q_6) - sin(q_6)*cos(q_4))*sin(q_2 + q_3) - sin(q_4)*sin(q_5)*cos(q_1)*cos(q_2 + q_3), (-sin(q_4)*sin(q_6)*cos(q_5) + cos(q_4)*cos(q_6))*cos(q_1) + sin(q_1)*sin(q_4)*sin(q_5), -(-sin(q_4)*si

In [37]:
d6d.inverseKinSymbols()

Matrix([
[                                                                                                                                                                             atan2(y, x)],
[                                                               atan2(z, sqrt(x**2 + y**2)) - atan2(3*sqrt(1 - (x**2/12 + y**2/12 + z**2/12 - 13/12)**2), x**2/4 + y**2/4 + z**2/4 - 5/4)],
[                                                                                                                                               acos(x**2/12 + y**2/12 + z**2/12 - 13/12)],
[                                                                                                                        -atan2(y, x) + atan2(sin(beta)*sin(gamma), sin(beta)*cos(gamma))],
[-acos(x**2/12 + y**2/12 + z**2/12 - 13/12) + acos(cos(beta)) - atan2(z, sqrt(x**2 + y**2)) + atan2(3*sqrt(1 - (x**2/12 + y**2/12 + z**2/12 - 13/12)**2), x**2/4 + y**2/4 + z**2/4 - 5/4)],
[                                                  

In [38]:
d6d.inverse(1.5,1.5,0,0,pi/2,0)

array([[ 0.78539816],
       [-1.62975605],
       [ 2.3579306 ],
       [-0.78539816],
       [ 0.84262177],
       [-3.14159265]])